# Mount drive and install packages

In [1]:
from google.colab import drive

In [9]:
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%cd /content/drive/MyDrive/dataset/

/content/drive/MyDrive/dataset


In [35]:
import time
import requests
!pip install simplejson
import simplejson as json
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Name input file and output file below

In [ ]:
filename = "postalcodes2.csv"
output = "latlongcoordinates.csv"

In [63]:
# Read in data, but removed the Maybank row because unable to find the postal code for just 'Maybank'

df_postcode=pd.read_csv(filename, dtype=str, encoding = 'latin-1')

In [64]:
# For loop to send postal codes to OneMAP API
# Input: Postal Codes as string format
# Output: Longitude and Latitude coordinates

for i in range(len(df_postcode)):
    if len(str(df_postcode['Postal Code'][i]))==5:
        df_postcode['Postal Code'][i]='0'+df_postcode['Postal Code'][i]
df_postcode['OnemapLongitude'] = pd.Series(0, index=df_postcode.index)
df_postcode['OnemapLatitude'] = pd.Series(0, index=df_postcode.index)       

start=time.time()
for i in range(len(df_postcode)):
    req=requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+df_postcode['Postal Code'][i]+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    jdata = json.loads(req.text)
    if jdata['found']>=1:
        df_postcode['OnemapLongitude'][i]=jdata['results'][0]['LONGITUDE']
        df_postcode['OnemapLatitude'][i]=jdata['results'][0]['LATITUDE']
    else:
        df_postcode['OnemapLongitude'][i]=float('nan')
        df_postcode['OnemapLatitude'][i]=float('nan')
df_postcode.to_csv(output)
print('Time Taken:',time.time()-start)

<ipython-input-64-525b0bc44c93>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_postcode['OnemapLongitude'][i]=jdata['results'][0]['LONGITUDE']
<ipython-input-64-525b0bc44c93>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_postcode['OnemapLatitude'][i]=jdata['results'][0]['LATITUDE']


Time Taken: 287.6481714248657
